In [1]:
import pandas as pd
# import tensorflow as tf
from packages import wrappers, ta

In [2]:
btcusd_df = wrappers.get_df_from_cryptowatch()

In [3]:
ta.add_ATR(btcusd_df, inplace=True)
ta.add_OBV(btcusd_df, inplace=True)
btcusd_df.dropna()

,Open,High,Low,Close,Volume,QuoteVolume,ATR_14,OBV
Date,,,,,,,,
2013-10-23,176.32429,194.20000,150.00000,150.0000,7.375877,0.000000e+00,10.384659,143.254748
2013-10-24,150.00000,203.45000,185.90287,203.4500,39.220000,0.000000e+00,13.460754,182.474748
2013-10-25,203.45000,207.30246,169.92010,193.0999,83.720975,0.000000e+00,15.169441,98.753773
2013-10-26,193.09990,186.80000,171.97000,186.0499,67.200000,0.000000e+00,15.595188,31.553773
2013-10-27,186.04990,185.80000,179.86560,184.5800,11.000000,0.000000e+00,14.922981,20.553773
...,...,...,...,...,...,...,...,...
2021-08-10,43845.60000,46484.10000,42827.70000,46307.1000,5654.101582,2.560590e+08,2458.373034,675192.386085
2021-08-11,46300.80000,46688.00000,44666.00000,45595.1000,3102.043540,1.412968e+08,2427.203531,672090.342545
2021-08-12,45606.90000,46736.40000,45373.50000,45571.7000,3504.532810,1.616380e+08,2351.181851,668585.809735


In [29]:
test = btcusd_df[['Close', 'QuoteVolume', 'Open']]
test['Open'] = test['Open'].shift(-1)
test.rename(columns={'Open': 'Target', 'QuoteVolume': 'Volume'})

,Close,Volume,Target
Date,,,
2013-10-08,123.61,0.000000e+00,123.61
2013-10-09,124.18,0.000000e+00,124.18
2013-10-10,123.84,0.000000e+00,123.84
2013-10-11,125.86,0.000000e+00,125.86
2013-10-13,127.00,0.000000e+00,127.00
...,...,...,...
2021-08-10,46307.10,2.560590e+08,46300.80
2021-08-11,45595.10,1.412968e+08,45606.90
2021-08-12,45571.70,1.616380e+08,45570.00


In [30]:
len(test)

2854

In [4]:
# feature_cols = [tf.feature_column.numeric_column(i) for i in ['Close', 'Volume', 'ATR_14', 'OBV']]

In [5]:
# estimator = tf.estimator.LinearRegressor(feature_columns=feature_cols, model_dir='./models/train')

ImportError: cannot import name 'parameter_server_strategy_v2' from 'tensorflow.python.distribute' (C:\Users\Plezo\anaconda3\lib\site-packages\tensorflow\python\distribute\__init__.py)